In [1]:
import math
import pandas as pd
import numpy as np
from operator import itemgetter

In [2]:
raw_terms=pd.read_csv('term.csv')
raw_data=pd.read_csv('text.csv')
raw_query=pd.read_csv('query.csv')

In [3]:
terms=list(raw_terms['term'])
query=list(raw_query['標題']+raw_query['內容'])
data=list(raw_data['標題']+raw_data['內容'])
print("共有 {} 個字".format(len(terms)))
print("共有 {} 筆query".format(len(query)))
print("共有 {} 筆data".format(len(data)))

del raw_terms,raw_query

共有 245 個字
共有 6 筆query
共有 2081 筆data


In [4]:
#前處理
word2id={}
id2word=[]
for i,term in enumerate(terms):
    word2id[term]=i
    id2word.append(term)
del terms

In [5]:
#建tf-idf
#算語料集和查詢集的tf值
#建245維向量，每維代表每個字在這篇文章中的tf值
query_tf=[]
for q in query:
    tmp=[]
    for term in id2word:
        tmp.append(q.count(term))
    query_tf.append(tmp)

data_tf=[]
for d in data:
    tmp=[]
    for term in id2word:
        tmp.append(d.count(term))
    data_tf.append(tmp)

In [6]:
#每維代表每個字的df值
df=[]
for term in id2word:
    df.append(0)
    for d in data:
        if term in d:
            df[word2id[term]]+=1

In [7]:
def idf(N,df):
    return math.log(N/df)

In [8]:
#美維代表每個字的tf-idf值
query_vec=[]
data_vec=[]
doc_num=len(data)

for tf in data_tf:
    tmp=[]
    for i in range(len(id2word)):
        tmp.append(tf[i]*idf(doc_num,df[i]))
    data_vec.append(tmp)
for tf in query_tf:
    tmp=[]
    for i in range(len(id2word)):
        tmp.append(tf[i]*idf(doc_num,df[i]))
    query_vec.append(tmp)

In [9]:
def get_expected_tf(doc_tf,df):
    result=[0]*len(doc_tf[0])
    for d in doc_tf:
        for i in range(len(d)):
            result[i]+=(d[i]/df[i])
    return result

In [10]:
def chi_square(O,E):
    sign=-1 if E>O else 1
    return(O-E)**2/E*sign

In [11]:
#美維代表每字的chi-square
query_vec=[]
data_vec=[]
doc_num=len(data)

expected_tf=get_expected_tf(data_tf,df)

for tf in data_tf:
    tmp=[]
    for i in range(len(id2word)):
        tmp.append(chi_square(tf[i],expected_tf[i]))
    data_vec.append(tmp)
    
for tf in query_tf:
    tmp=[]
    for i in range(len(id2word)):
        tmp.append(chi_square(tf[i],expected_tf[i]))
    query_vec.append(tmp)

In [15]:
#算相似度
def cal_score(q_vec,d_vec):
    q_vec=np.array(q_vec)
    d_vec=np.array(d_vec)
    norm_q=np.sqrt(np.sum(q_vec**2))
    norm_d=np.sqrt(np.sum(d_vec**2))
    
    return np.dot(q_vec,d_vec)/(norm_q*norm_d)

In [16]:
#拿查詢集的第一筆資料來和語料集做相似度
q_id=0
data_score=[]
for i , d in enumerate(data_vec):
    data_score.append((i,cal_score(query_vec[q_id],d)))

In [17]:
#排序
data_score.sort(key=itemgetter(1),reverse=True)
#取前三名
for d in data_score[:3]:
    print(raw_data.iloc[d[0]])

編號                                                   65
類別                                                 鴻海新聞
時間                                       1/15/2016 8:30
標題                                鴻海傳加碼至7,000億日圓搶親 夏普飆漲
內容    日本讀賣新聞報導，鴻海考慮提高投資夏普的金額，一口氣從5,000億日圓加碼至7,000億日圓...
Name: 64, dtype: object
編號                                                  733
類別                                                 鴻海新聞
時間                                       2/5/2016 18:37
標題                                         夏普否認給鴻海優先談判權
內容    彭博資訊報導，日本夏普公司5日表示，否認給予鴻海優先交涉權的報導，指出鴻海作為優先合作夥伴的...
Name: 732, dtype: object
編號                                                 1030
類別                                                 鴻海新聞
時間                                       3/10/2016 2:40
標題                                    鴻夏戀／明天311 不會有重大宣布
內容    日本媒體原先預期鴻夏將在本周有結果，不過昨（9）日外電指出，適逢明（11）日是日本東北311...
Name: 1029, dtype: object
